<a href="https://colab.research.google.com/github/hamletbatista/inbound/blob/master/CheatSheet_Exporting_to_Sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exporting-to-Sheets 101 with Python

Tool: https://colab.research.google.com/notebooks/data_table.ipynb


Libraries

1. Pandas
2. Gspread
3. Gspread-pandas

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!ls

sample_data  themes_seo.pkl


In [ ]:
import pickle

In [ ]:
with open('themes_seo.pkl', 'rb') as f:

  themes_seo = pickle.load(f)

In [ ]:
themes_seo.keys()

dict_keys(['Prestige', 'Reach', 'Avenue', 'Empire', 'Focal', 'Mr Parker', 'Label', 'Debut', 'Grid', 'Icon', 'Boost', 'Capital', 'Cascade', 'Flow', 'Minimal', 'Editorial', 'Streamline', 'Mobilia', 'District', 'Editions', 'Testament', 'Artisan', 'Canopy', 'theme', 'Context', 'Expression', 'Narrative', 'Galleria', 'Pacific', 'Supply', 'Symmetry', 'Startup', 'Emerge', 'Handy', 'Loft', 'Story', 'Trademark', 'Vantage', 'Impulse', 'Kingdom', 'Envy', 'Maker', 'Venture', 'Pipeline', 'Responsive', 'Split', 'Motion', 'Lorenza', 'Ira', 'Alchemy', 'Masonry', 'Boundless', 'Brooklyn', 'Blockshop', 'Palo Alto', 'Vogue', 'ShowTime', 'Broadcast', 'Parallax', 'Providence', 'Warehouse', 'Simple', 'California', 'Sunrise', 'Launch', 'Retina', 'Colors', 'Kagami', 'Modular', 'Venue', 'Fashionopolism', 'Atlantic', 'Showcase', 'Express'])

In [ ]:

!pip install git+https://github.com/aiguofer/gspread-pandas.git


In [ ]:
# Authenticate (copy & paste key as detailed)
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials

In [ ]:
from gspread_pandas import Spread, Client

In [ ]:
creds = GoogleCredentials.get_application_default()

In [ ]:
spread = Spread("INBOUND2020", creds=creds)


In [ ]:
!ls

sample_data  themes_seo.pkl


In [ ]:
themes_seo["Prestige"]

[{'seo_score': 0.76},
 {'audit': 'is-crawlable',
  'audit_description': "Search engines are unable to include your pages in search results if they don't have permission to crawl them. [Learn more](https://web.dev/is-crawable).",
  'audit_score': 0,
  'audit_title': 'Page is blocked from indexing'},
 {'audit': 'meta-description',
  'audit_description': 'Meta descriptions may be included in search results to concisely summarize page content. [Learn more](https://web.dev/meta-description).',
  'audit_score': 0,
  'audit_title': 'Document does not have a meta description'},
 {'audit': 'hreflang',
  'audit_description': 'hreflang links tell search engines what version of a page they should list in search results for a given language or region. [Learn more](https://web.dev/hreflang).',
  'audit_score': 1,
  'audit_title': 'Document has a valid `hreflang`'},
 {'audit': 'canonical',
  'audit_description': 'Canonical links suggest which URL to show in search results. [Learn more](https://web.de

In [ ]:
import pandas as pd

In [ ]:
#https://colab.research.google.com/notebooks/data_table.ipynb
%load_ext google.colab.data_table


In [ ]:
seo_score = themes_seo["Prestige"][0]

In [ ]:
seo_score

{'seo_score': 0.76}

In [ ]:
df = pd.DataFrame(themes_seo["Prestige"][1:])

In [ ]:
df

,audit,audit_title,audit_description,audit_score
0,is-crawlable,Page is blocked from indexing,Search engines are unable to include your page...,0.00
1,meta-description,Document does not have a meta description,Meta descriptions may be included in search re...,0.00
2,hreflang,Document has a valid `hreflang`,hreflang links tell search engines what versio...,1.00
3,canonical,Document has a valid `rel=canonical`,Canonical links suggest which URL to show in s...,1.00
4,document-title,Document has a `<title>` element,The title gives screen reader users an overvie...,1.00
5,image-alt,Image elements have `[alt]` attributes,"Informative elements should aim for short, des...",1.00
6,structured-data,Structured data is valid,Run the [Structured Data Testing Tool](https:/...,NaN
7,tap-targets,Tap targets are not sized appropriately,Interactive elements like buttons and links sh...,0.87


In [ ]:
len(themes_seo)

74

### Adding theme data to Sheet

In [ ]:
for theme_name, audits in themes_seo.items():
  seo_score = audits[0]
  df = pd.DataFrame(audits[1:])

  spread.df_to_sheet(df, index=False, sheet=theme_name, start='A1', replace=True)



In [ ]:
themes_seo.keys()

dict_keys(['Prestige', 'Reach', 'Avenue', 'Empire', 'Focal', 'Mr Parker', 'Label', 'Debut', 'Grid', 'Icon', 'Boost', 'Capital', 'Cascade', 'Flow', 'Minimal', 'Editorial', 'Streamline', 'Mobilia', 'District', 'Editions', 'Testament', 'Artisan', 'Canopy', 'theme', 'Context', 'Expression', 'Narrative', 'Galleria', 'Pacific', 'Supply', 'Symmetry', 'Startup', 'Emerge', 'Handy', 'Loft', 'Story', 'Trademark', 'Vantage', 'Impulse', 'Kingdom', 'Envy', 'Maker', 'Venture', 'Pipeline', 'Responsive', 'Split', 'Motion', 'Lorenza', 'Ira', 'Alchemy', 'Masonry', 'Boundless', 'Brooklyn', 'Blockshop', 'Palo Alto', 'Vogue', 'ShowTime', 'Broadcast', 'Parallax', 'Providence', 'Warehouse', 'Simple', 'California', 'Sunrise', 'Launch', 'Retina', 'Colors', 'Kagami', 'Modular', 'Venue', 'Fashionopolism', 'Atlantic', 'Showcase', 'Express'])

In [ ]:
new_format = list()

for name in themes_seo.keys():

  if name == "theme":
    continue

  theme_info = dict()

  for row in themes_seo[name][1:]:
    # Repeat for each audit
    score = themes_seo[name][0]
    theme_info["name"] = name
    theme_info["score"] = score["seo_score"]
    
    #theme_info[k] = v
    audit_name = row["audit"]
    audit_title = row["audit_title"]
    audit_description = row["audit_description"]
    audit_score = row["audit_score"]

    theme_info[audit_name] = audit_title #audit_title, audit_description, audit_score
    theme_info[f"{audit_name}_description"] = audit_description
    theme_info[f"{audit_name}_score"] = audit_score

  new_format.append(theme_info)
  

In [ ]:
new_format

In [ ]:
df = pd.DataFrame(new_format)

In [ ]:
df

,name,score,is-crawlable,is-crawlable_description,is-crawlable_score,meta-description,meta-description_description,meta-description_score,hreflang,hreflang_description,hreflang_score,canonical,canonical_description,canonical_score,document-title,document-title_description,document-title_score,image-alt,image-alt_description,image-alt_score,structured-data,structured-data_description,structured-data_score,tap-targets,tap-targets_description,tap-targets_score
0,Prestige,0.76,Page is blocked from indexing,Search engines are unable to include your page...,0,Document does not have a meta description,Meta descriptions may be included in search re...,0,Document has a valid `hreflang`,hreflang links tell search engines what versio...,1,Document has a valid `rel=canonical`,Canonical links suggest which URL to show in s...,1,Document has a `<title>` element,The title gives screen reader users an overvie...,1,Image elements have `[alt]` attributes,"Informative elements should aim for short, des...",1,Structured data is valid,Run the [Structured Data Testing Tool](https:/...,None,Tap targets are not sized appropriately,Interactive elements like buttons and links sh...,0.87
1,Reach,0.76,Page isn’t blocked from indexing,Search engines are unable to include your page...,1,Document does not have a meta description,Meta descriptions may be included in search re...,0,Document has a valid `hreflang`,hreflang links tell search engines what versio...,1,Document has a valid `rel=canonical`,Canonical links suggest which URL to show in s...,1,Document has a `<title>` element,The title gives screen reader users an overvie...,1,Image elements do not have `[alt]` attributes,"Informative elements should aim for short, des...",0,Structured data is valid,Run the [Structured Data Testing Tool](https:/...,None,Tap targets are not sized appropriately,Interactive elements like buttons and links sh...,0.82
2,Avenue,0.91,Page isn’t blocked from indexing,Search engines are unable to include your page...,1,Document has a meta description,Meta descriptions may be included in search re...,1,Document has a valid `hreflang`,hreflang links tell search engines what versio...,1,Document has a valid `rel=canonical`,Canonical links suggest which URL to show in s...,1,Document has a `<title>` element,The title gives screen reader users an overvie...,1,Image elements have `[alt]` attributes,"Informative elements should aim for short, des...",1,Structured data is valid,Run the [Structured Data Testing Tool](https:/...,None,Tap targets are not sized appropriately,Interactive elements like buttons and links sh...,0.84
3,Empire,0.89,Page isn’t blocked from indexing,Search engines are unable to include your page...,1,Document has a meta description,Meta descriptions may be included in search re...,1,Document has a valid `hreflang`,hreflang links tell search engines what versio...,1,Document has a valid `rel=canonical`,Canonical links suggest which URL to show in s...,1,Document has a `<title>` element,The title gives screen reader users an overvie...,1,Image elements have `[alt]` attributes,"Informative elements should aim for short, des...",1,Structured data is valid,Run the [Structured Data Testing Tool](https:/...,None,Tap targets are not sized appropriately,Interactive elements like buttons and links sh...,0.61
4,Focal,0.81,Page isn’t blocked from indexing,Search engines are unable to include your page...,1,Document does not have a meta description,Meta descriptions may be included in search re...,0,Document has a valid `hreflang`,hreflang links tell search engines what versio...,1,Document has a valid `rel=canonical`,Canonical links suggest which URL to show in s...,1,Document has a `<title>` element,The title gives screen reader users an overvie...,1,Image elements have `[alt]` attributes,"Informative elements should aim for short, des...",1,Structured data is valid,Run the [Structured Data Testing Tool](https:/...,None,Tap targets are not sized appropriately,Interactive el

In [ ]:
spread = Spread("INBOUND2020", creds=creds)


In [ ]:
spread.df_to_sheet(df, index=False, sheet="Themes SEO audits", start='A1', replace=True)
